## Упражнения из 8 семинара

In [28]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

data_train = tfds.load(name="fashion_mnist", split="train").prefetch(60000).cache()
data_test  = tfds.load(name="fashion_mnist", split="test" ).prefetch(10000).cache()

### Ex. 1

Move the preprocessing part out of the model into the dataset pipeline (using the `map` method).

In [54]:
num_hidden = 64

model = tf.keras.Sequential([
    

  # Densely connected hidden layer
  tf.keras.layers.Dense(num_hidden, activation='relu'),

  # Output layer (note there's no activation, not even softmax:
  # this layer will output logits, there are pre-implemented
  # loss functions in `keras` to work with such outputs)
  tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),metrics = 'accuracy')

In [55]:
print(data_train)

<CacheDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>


При применении `reshape` к изображению возникала ошибка `TypeError: 'x' and 'y' must have the same dtype, got tf.uint8 != tf.float32`. Не знаю, почему подобной ошибки не возникало в семинаре, но лучшее что я придумал - изменить dtype командой `tf.cast`.

In [56]:
def reshape(x):  
    x_new = tf.reshape(tf.cast(x['image'],dtype=tf.float32) / 255., (-1, 28*28))
    return(x_new,x['label'])

In [57]:
batch_size = 1024
model.fit(x=data_train.shuffle(60000).batch(batch_size).map(reshape),
          epochs=10,
          validation_data=data_test.batch(4096).map(reshape))

Epoch 1/10
59/59 [==============================] - 1s 5ms/step - loss: 0.9644 - accuracy: 0.6814 - val_loss: 0.6410 - val_accuracy: 0.7752
Epoch 2/10
59/59 [==============================] - 0s 5ms/step - loss: 0.5580 - accuracy: 0.8127 - val_loss: 0.5407 - val_accuracy: 0.8140
Epoch 3/10
59/59 [==============================] - 0s 5ms/step - loss: 0.4878 - accuracy: 0.8359 - val_loss: 0.4954 - val_accuracy: 0.8275
Epoch 4/10
59/59 [==============================] - 0s 4ms/step - loss: 0.4529 - accuracy: 0.8465 - val_loss: 0.4736 - val_accuracy: 0.8321
Epoch 5/10
59/59 [==============================] - 0s 5ms/step - loss: 0.4299 - accuracy: 0.8543 - val_loss: 0.4526 - val_accuracy: 0.8431
Epoch 6/10
59/59 [==============================] - 0s 5ms/step - loss: 0.4127 - accuracy: 0.8590 - val_loss: 0.4453 - val_accuracy: 0.8453
Epoch 7/10
59/59 [==============================] - 0s 4ms/step - loss: 0.4008 - accuracy: 0.8623 - val_loss: 0.4317 - val_accuracy: 0.8482
Epoch 8/10
59/59 [==

Кажется, получилось!

### Ex. 2

Train a model with more layers.

In [65]:
num_hidden_1 = 64
num_hidden_2 = 128
num_hidden_3 = 32

model = tf.keras.Sequential([
    
  tf.keras.layers.Lambda(
      lambda x: tf.reshape(x / 255., (-1, 28*28)),
      input_shape=(28, 28, 1), name='preprocessing'
  ),
    
  # Densely connected hidden layer
  tf.keras.layers.Dense(num_hidden_1, activation='relu'),
  tf.keras.layers.Dense(num_hidden_2, activation='relu'),
  tf.keras.layers.Dense(num_hidden_3, activation='relu'),

  # Output layer (note there's no activation, not even softmax:
  # this layer will output logits, there are pre-implemented
  # loss functions in `keras` to work with such outputs)
  tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),metrics = 'accuracy')

In [66]:
def unwrap(x):
    return(x['image'],x['label'])

In [67]:
batch_size = 1024
model.fit(x=data_train.map(unwrap).shuffle(60000).batch(batch_size),
          epochs=10,
          validation_data=data_test.map(unwrap).batch(4096))

Epoch 1/10
59/59 [==============================] - 1s 7ms/step - loss: 1.0199 - accuracy: 0.6378 - val_loss: 0.6106 - val_accuracy: 0.7863
Epoch 2/10
59/59 [==============================] - 0s 6ms/step - loss: 0.5141 - accuracy: 0.8226 - val_loss: 0.4971 - val_accuracy: 0.8264
Epoch 3/10
59/59 [==============================] - 0s 6ms/step - loss: 0.4459 - accuracy: 0.8459 - val_loss: 0.4573 - val_accuracy: 0.8403
Epoch 4/10
59/59 [==============================] - 0s 6ms/step - loss: 0.4093 - accuracy: 0.8575 - val_loss: 0.4359 - val_accuracy: 0.8519
Epoch 5/10
59/59 [==============================] - 0s 6ms/step - loss: 0.3895 - accuracy: 0.8629 - val_loss: 0.4226 - val_accuracy: 0.8524
Epoch 6/10
59/59 [==============================] - 0s 6ms/step - loss: 0.3779 - accuracy: 0.8657 - val_loss: 0.4100 - val_accuracy: 0.8571
Epoch 7/10
59/59 [==============================] - 0s 6ms/step - loss: 0.3600 - accuracy: 0.8724 - val_loss: 0.4105 - val_accuracy: 0.8552
Epoch 8/10
59/59 [==

In [68]:
X_batches = []
y_batches = []
for batch in data_test.batch(4096):
  X_batches.append(batch['image'].numpy().astype('float32'))
  y_batches.append(batch['label'].numpy())

X_test = np.concatenate(X_batches)
y_test = np.concatenate(y_batches)
def check_accuracy(model):
  return (model(X_test).numpy().argmax(axis=1) == y_test).mean()



In [69]:
check_accuracy(model)

0.8612

Значение чуть улучшилось по сравнению с 0.8508!

In [70]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocessing (Lambda)      (None, 784)               0         
                                                                 
 dense_26 (Dense)            (None, 64)                50240     
                                                                 
 dense_27 (Dense)            (None, 128)               8320      
                                                                 
 dense_28 (Dense)            (None, 32)                4128      
                                                                 
 dense_29 (Dense)            (None, 10)                330       
                                                                 
Total params: 63,018
Trainable params: 63,018
Non-trainable params: 0
_________________________________________________________________
